In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import pandas as pd
import scipy
from scipy.optimize import minimize
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import linregress
import statsmodels.api as sm

In [ ]:
iid_equal = pd.read_csv('all_subject_equal.csv',sep ="\t", header=0)
iid_n_equal = pd.read_csv('all_subject_n_equal.csv',sep ="\t", header=0)

In [ ]:
iid_n_equal.columns

In [ ]:
def path_responces (path):
    grouped_data = path.groupby(['index_traj', 'condition', 'mean_side']).agg(
    path=pd.NamedAgg(column='path', aggfunc='first'),
    last_cond=pd.NamedAgg(column='last_cond', aggfunc='first'),
    index_old=pd.NamedAgg(column='index_old', aggfunc='first'),
    mean_val=pd.NamedAgg(column='mean_val', aggfunc='first'),
    mean_last5=pd.NamedAgg(column='mean_last5', aggfunc='first'),
    mean_last15=pd.NamedAgg(column='mean_last15', aggfunc='first'),
    sd=pd.NamedAgg(column='sd', aggfunc='first'),
    last_visibl_side=pd.NamedAgg(column='last_visibl_side', aggfunc='first'),
    f_missing_pos=pd.NamedAgg(column='f_missing_pos', aggfunc='first'),
    last_visibl_center_dist_deg=pd.NamedAgg(column='last_visibl_center_dist[deg]', aggfunc='first'),
    correct=pd.NamedAgg(column='accuracy', aggfunc=lambda x: (x == 1).sum()),
    error=pd.NamedAgg(column='accuracy', aggfunc=lambda x: (x == 0).sum()),
    acc=pd.NamedAgg(column='accuracy', aggfunc=lambda x: np.round(x.mean(), 3)),
    RT=pd.NamedAgg(column='RT', aggfunc=lambda x: np.round(x.mean(), 3))).reset_index()
    grouped_data['mean_abs'] = grouped_data['mean_val'].apply(abs)
    group_sd = []
    for sd in grouped_data['sd']:
        if sd < 3.4:
            group_sd.append(1)
        elif 3.4 <= sd < 5.3:
            group_sd.append(2)
        else:  # Assuming sd >= 5.5
            group_sd.append(3)
    grouped_data['group_sd'] = group_sd
    grouped_data['mean/std'] = round( abs(grouped_data['mean_val']/ grouped_data['sd']),3)
    # Calculating the difference between mean_abs and JND for each group
    diff_list = []
    # jnd_values = {1: 0.386, 2: 0.581, 3: 0.654}
    # jnd_values = {1: 1.3, 2: 1.3, 3: 1.3}
    jnd_values = {1: 0.6, 2: 0.9, 3: 1.3}
    for i in range(grouped_data.shape[0]):
        group = grouped_data['group_sd'][i]
        jnd = jnd_values[group]
    #     diff = round(abs(grouped_data['mean_abs'][i] - jnd), 3)
        diff = round(grouped_data['mean_abs'][i] - jnd, 3)
        diff_list.append(diff)

    grouped_data['mu-jnd'] = diff_list

    return grouped_data
    

In [ ]:
grouped_iid_n_equal=path_responces (iid_n_equal)
grouped_iid_equal=path_responces (iid_equal)
grouped_iid_n_equal

In [ ]:
def make_LR_mu_std(grouped_data):
    models = []

    grouped = grouped_data.groupby(['path',])

    for group_name, group_data in grouped:
        X = sm.add_constant(group_data['mean/std'])  
        y = group_data['acc']  
        model = sm.OLS(y, X).fit()  
        models.append((group_name, model))  

    for group_name, model in models:
        print(f"Group : {group_name}, {grouped_data['last_cond']}")
        print(model.summary())
        print("\n")
    return models

In [ ]:
models_n_equal = make_LR_mu_std(grouped_iid_n_equal)
models_equal = make_LR_mu_std(grouped_iid_equal)

In [ ]:
def make_LR(grouped_data):
    models = []

    grouped = grouped_data.groupby(['path',])

    for group_name, group_data in grouped:
        X = sm.add_constant(group_data['mu-jnd'])  
        y = group_data['acc']  
        model = sm.OLS(y, X).fit()  
        models.append((group_name, model))  

    for group_name, model in models:
        print(f"Group : {group_name}, {grouped_data['last_cond']}")
        print(model.summary())
        print("\n")
    return models

In [ ]:
models_n_equal = make_LR(grouped_iid_n_equal)
models_n_equal = make_LR(grouped_iid_equal)

In [ ]:
def plotting (grouped_data_1,grouped_data_2):
#     plt.subplot(1,1,1)
    sns.lmplot(x='mu-jnd',
           y='acc',
           data=grouped_data_1,
           hue='path',
           ci=None,
           scatter_kws={'alpha': 0.5})
    plt.title('IID EQUAL: mu-jnd vs acc ')
    plt.xlabel('mu-jnd difference')
    plt.ylabel('mean acc')
    # Linear regression by mean_cond groups
    linear_regression_by_group = grouped_data_1.groupby('path').apply(
        lambda group: linregress(group['mu-jnd'], group['acc']))
    print("IID EQUAL:")
    for group, result in zip(linear_regression_by_group.index,
                             linear_regression_by_group):
        slope, intercept, r_value, p_value, std_err = result
        
        print(
            f"Group: {group}, Slope = {round(slope,3)}, Intercept = {round(intercept,3)}, $R^2$={r_value**2:.2f}, P-value = {round(p_value,3)}, Std_err = {round(std_err,3)}"
        )
    sns.lmplot(x='mu-jnd',
           y='acc',
           data=grouped_data_2,
           hue='path',
           ci=None,
           scatter_kws={'alpha': 0.5})
    plt.title('IID NOT EQUAL: mu-jnd vs acc ')
    plt.xlabel('mu-jnd difference')
    plt.ylabel('mean acc')
    # Linear regression by mean_cond groups
    linear_regression_by_group = grouped_data_2.groupby('path').apply(
        lambda group: linregress(group['mu-jnd'], group['acc']))
    print("IID NOT EQUAL:")
    for group, result in zip(linear_regression_by_group.index,
                             linear_regression_by_group):
        slope, intercept, r_value, p_value, std_err = result
        
        print(
            f"Group: {group}, Slope = {round(slope,3)}, Intercept = {round(intercept,3)}, $R^2$={r_value**2:.2f}, P-value = {round(p_value,3)}, Std_err = {round(std_err,3)}"
        )
        

In [ ]:
plotting (grouped_iid_equal,grouped_iid_n_equal)

In [ ]:
# Scatter plot with regression line
sns.lmplot(x='mean/std', y='acc', data=grouped_iid_equal, ci=None, scatter_kws={'alpha':0.5})
plt.title('IID EQUAL: mean/std vs acc')
plt.xlabel('mean/std')
plt.ylabel('acc ')
plt.show()

slope, intercept, r_value, p_value, std_err = linregress(grouped_iid_equal['mean/std'], grouped_iid_equal['acc'])
print(f"IID EQUAL: slope = {round(slope,3)}, intercept = {round(intercept,3)}, r_value = {round(r_value,3)}, p_value = {round(p_value,3)}, std_err = {round(std_err,3)}")

sns.lmplot(x='mean/std', y='acc', data=grouped_iid_n_equal, ci=None, scatter_kws={'alpha':0.5})
plt.title('IID NOT EQUAL: mu-jndvs acc')
plt.xlabel('mean/std')
plt.ylabel('acc ')
plt.show()

slope, intercept, r_value, p_value, std_err = linregress(grouped_iid_n_equal['mean/std'], grouped_iid_n_equal['acc'])
print(f"IID NOT EQUAL: slope = {round(slope,3)}, intercept = {round(intercept,3)}, r_value = {round(r_value,3)}, p_value = {round(p_value,3)}, std_err = {round(std_err,3)}")